YOLO V5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content

/content


In [ ]:
import yaml
import torch
from IPython.display import Image, clear_output
!git clone https://github.com/ultralytics/yolov5
% cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 12173, done.
remote: Total 12173 (delta 0), reused 0 (delta 0), pack-reused 12173
Receiving objects: 100% (12173/12173), 11.96 MiB | 8.13 MiB/s, done.
Resolving deltas: 100% (8428/8428), done.
/content/yolov5
     |████████████████████████████████| 596 kB 4.2 MB/s 


In [ ]:
!pwd

/content/yolov5


In [ ]:
# %mkdir /content/yolov5/pothole
%cd /content/yolov5

/content/yolov5


In [ ]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 4.2 MB/s 
     |████████████████████████████████| 178 kB 30.7 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 138 kB 88.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=067e6d35bd6b4b5c8b9bfaca10c0c9604cfbb4eb69a226e207faf617edb8f924
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=00f6842673496bf8c6538a780ea0a08712734cca731d26ba4e8d85a85aba7b6b
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="dXP7LFrMmhFIk8sby9ZT")
project = rf.workspace("multiplication").project("test-5rv3v")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to TEST-2 in yolov5pytorch:: 100%|██████████| 19212/19212 [00:02<00:00, 8321.65it/s]


In [ ]:
!ls

CONTRIBUTING.md  export.py   models	       setup.cfg  tutorial.ipynb
data		 hubconf.py  README.md	       TEST-2	  utils
detect.py	 LICENSE     requirements.txt  train.py   val.py


학습

In [ ]:
from glob import glob

train_img_ilst = glob('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/train/images/*.jpg')
test_img_ilst = glob('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/test/images/*.jpg')
valid_img_ilst = glob('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/valid/images/*.jpg')
print(len(train_img_ilst))
print(len(test_img_ilst))
print(len(valid_img_ilst))

6720
960
1920


YAML은 구성 파일 작성에 자주 사용되는 데이터 직렬화 언어

In [ ]:
import yaml

with open('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/train.text', 'w') as f:
  f.write('\n'.join(train_img_ilst)+'\n')
with open('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/test.text', 'w') as f:
  f.write('\n'.join(test_img_ilst)+'\n')
with open('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/valid.text', 'w') as f:
  f.write('\n'.join(valid_img_ilst)+'\n')

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

In [ ]:
# cat 두 개 이상의 파일을 연결(concatenate)해서 출력

%cat /content/yolov5/Strawberry-Disease-Detection-Dataset-4/data.yaml

names: [Angular Leafspot, Anthracnose Fruit Rot, Blossom Blight, Gray Mold, Leaf Spot,
  Powdery Mildew Fruit, Powdery Mildew Leaf]
nc: 7
train: TEST-2/train/images
val: TEST-2/valid/images


In [ ]:
%%writetemplate /content/yolov5/Strawberry-Disease-Detection-Dataset-4/data.yaml
train: ./Strawberry-Disease-Detection-Dataset-4/train/images
test : ./Strawberry-Disease-Detection-Dataset-4/test/images
val: ./Strawberry-Disease-Detection-Dataset-4/valid/images

nc: 7
names: ['Angular Leafspot', 'Anthracnose Fruit Rot', 'Blossom Blight', 'Gray Mold', 'Leaf Spot', 'Powdery Mildew Fruit', 'Powdery Mildew Leaf']

In [ ]:
%cat /content/yolov5/Strawberry-Disease-Detection-Dataset-4/data.yaml

train: ./Strawberry-Disease-Detection-Dataset-4/train/images
test : ./Strawberry-Disease-Detection-Dataset-4/test/images
val: ./Strawberry-Disease-Detection-Dataset-4/valid/images

nc: 7
names: ['Angular Leafspot', 'Anthracnose Fruit Rot', 'Blossom Blight', 'Gray Mold', 'Leaf Spot', 'Powdery Mildew Fruit', 'Powdery Mildew Leaf']

In [ ]:
# 모델 구성

with open('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/data.yaml', 'r') as stream: # stream 입출력
  num_classes = str(yaml.safe_load(stream)['nc'])

num_classes

'7'

In [ ]:
%cat /content/yolov5/models/yolov5l.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]], 

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5l.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat /content/yolov5/models/custom_yolov5l.yaml


# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 7  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]], 

학습(Training)
- img : 입력 이미지 크기 정의
- batch : 배치 크기 결정
- epochs : 학습 기간 개수 정의
- data : yaml 파일 경로
- cfg : 모델 구성 지정
- weights : 가중치에 대한 경로 지정
- name : 결과 이름
- nosave : 최종 체크포인트만 저장
- cache : 빠른 학습을 위한 이미지 캐시


In [ ]:
# python train.py --data coco.yaml --cfg yolov5s.yaml --weights '' --batch-size 64
#                                        yolov5m                   --batch-size 40                           
#                                        yolov5l                   --batch-size 24
#                                        yolov5x                   --batch-size 16

In [ ]:
%time
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 500 --data ./Strawberry-Disease-Detection-Dataset-4/data.yaml --cfg ./models/custom_yolov5l.yaml --weights '' --name strawberry_results --cache

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs
/content/yolov5
train: weights=, cfg=./models/custom_yolov5l.yaml, data=./Strawberry-Disease-Detection-Dataset-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=strawberry_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-244-g47233e1 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_

yaml 모델, data yaml, best.pt

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!ls /content/yolov5/runs/train/strawberry_results4

ls: cannot access '/content/yolov5/runs/train/strawberry_results4': No such file or directory


In [ ]:
Image(filename='/content/yolov5/runs/train/strawberry_results4/train_batch0.jpg', width=1000)

FileNotFoundError: ignored

In [ ]:
Image(filename='/content/yolov5/runs/train/strawberry_results4/val_batch0_labels.jpg', width=1000)

In [ ]:
!pwd

In [ ]:
# 모델 내보내기

%mkdir /content/drive/MyDrive/MKS/project3/strawberry_test_v5l  
%cp /content/yolov5/runs/train/strawberry_results4/weights/best.pt /content/drive/MyDrive/MKS/project3/strawberry_test_v5l

In [ ]:
%cp /content/yolov5/Strawberry-Disease-Detection-Dataset-4/data.yaml /content/drive/MyDrive/MKS/project3/strawberry_test_v5l

In [ ]:
%cp /content/yolov5/models/custom_yolov5l.yaml /content/drive/MyDrive/MKS/project3/strawberry_test_v5l

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/3차_프로젝트/version2/YoloL93.pt --img 660 --conf 0.25 --source /content/Strawberry_-last-8/train/images --conf 0.15 --line-thickness 1

detect: weights=['/content/drive/MyDrive/3차_프로젝트/version2/YoloL93.pt'], source=/content/Strawberry_-last-8/train/images, data=data/coco128.yaml, imgsz=[660, 660], conf_thres=0.15, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-244-g47233e1 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 252, in <module>
    main(opt)
  File "/content/yolov5/detect.py", line 247, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolov5/detect.py", line 92, in run
    model = Detect

In [ ]:
import glob
import random

image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp7/*.jpg'))
display(Image(filename=image_name)) 

IndexError: ignored

In [ ]:
!python val.py --weights /content/drive/MyDrive/MKS/project3/strawberry_test_v5l/best.pt --data /content/yolov5/Strawberry_-last-5/data.yaml --img 640

In [ ]:
!python detect.py --weights /content/drive/MyDrive/MKS/project3/strawberry_test_v5l/best.pt --img 1080 --conf 0.25 --source /content/yolov5/Strawberry_-last-5/test/images

detect: weights=['/content/drive/MyDrive/MKS/project3/strawberry_test_v5l/best.pt'], source=/content/yolov5/Strawberry_-last-5/test/images, data=data/coco128.yaml, imgsz=[1080, 1080], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-236-gdcf8073 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

Traceback (most recent call last):
  File "detect.py", line 252, in <module>
    main(opt)
  File "detect.py", line 247, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 92, in run
    model = DetectMultiBackend(weights, device=devi